In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import os
import glob
from torch.utils.data import DataLoader

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data-science-dating-project/df_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58822 entries, 0 to 58821
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  58822 non-null  int64  
 1   status               58822 non-null  object 
 2   sex                  58822 non-null  object 
 3   orientation          58822 non-null  object 
 4   diet                 58822 non-null  object 
 5   drinks               58822 non-null  object 
 6   drugs                58822 non-null  object 
 7   education            58822 non-null  object 
 8   ethnicity            58822 non-null  object 
 9   height               58822 non-null  float64
 10  job                  58822 non-null  object 
 11  location             58822 non-null  object 
 12  offspring            58822 non-null  object 
 13  pets                 58822 non-null  object 
 14  religion             58822 non-null  object 
 15  smokes               58822 non-null 

NOTE: "speaks_original" contains a non-human language "lisp". Please remove before getting into ML modeling.

In [ ]:
df.sample(3)

,age,status,sex,orientation,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,smokes,speaks,speaks_original,combined_essay_cols,cleaned_essays
7572,27,single,m,straight,unknown,socially,never,college/university_graduated,unknown,72.0,science / tech / engineering,"berkeley, california",unknown,likes pets,other,no,multilingual,"english, spanish",i am the key master. i won a race amongst mil...,key master race amongst million berkeley nativ...
6465,25,single,m,straight,anything,socially,never,college/university_studying,unknown,76.0,hospitality / travel,"palo alto, california","no Kids, might want",unknown,christianity,no,monolingual,english,,NaN
43515,41,single,f,straight,unknown,socially,never,masters_graduated,mixed,66.0,computer / hardware / software,"san francisco, california",unknown,unknown,other,no,multilingual,"english, farsi, greek, french","are you jumping out of a plane one day, and tr...",jumping plane one day trekking exotic mountain...


## BERT Embeddings Approach

Sentence embeddings allow us to convert sentences into numerical vectors, capturing their meaning in a way that's easy for machine learning models to use.

all-MiniLM-L6-v2 is a sentence-transformer model that is great for tasks like semantic search, clustering, or identifying sentence similarity.

In [ ]:
# moving cleaned_essays column to a new df
bert_df = pd.DataFrame(df['cleaned_essays'])
df = df.drop('cleaned_essays', axis=1)

In [ ]:
bert_df.sample(5)

,cleaned_essays
9639,somewhat shy funloving person like try new thi...
5342,im sensitive intelligent widelyread playful ma...
50574,hmmmwhat put let see love laugh like people se...
7662,new la pretty busy starting business friend su...
7143,laidback easy going guy sport business fanatic...


In [ ]:
bert_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58822 entries, 0 to 58821
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cleaned_essays  57169 non-null  object
dtypes: object(1)
memory usage: 459.7+ KB


In [ ]:
SAVE_DIR = "/content/drive/My Drive/bert_embeddings/"
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
# # Function to generate & save embeddings batch-by-batch

# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name).to(device)

# def bert_embeddings_colab(text_list, batch_size=8):
#     """
#     Generate BERT embeddings in batches and save to Google Drive to prevent memory overflow.
#     """
#     embeddings_list = []

#     for i in range(0, len(text_list), batch_size):
#         batch = text_list[i : i + batch_size]

#         # Tokenize batch & move to GPU
#         encoded_inputs = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

#         # Compute embeddings
#         with torch.no_grad():
#             model_output = model(**encoded_inputs)

#         # Mean pooling
#         token_embeddings = model_output[0]
#         input_mask_expanded = encoded_inputs['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
#         batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#         # Normalize embeddings
#         batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)

#         # Convert to NumPy & store in list
#         embeddings_list.append(batch_embeddings.cpu().numpy())

#         # Save every 5 batches to avoid memory issues
#         if (i // batch_size + 1) % 5 == 0:
#             np.save(os.path.join(SAVE_DIR, f"embeddings_batch_{i//batch_size}.npy"), np.vstack(embeddings_list))
#             embeddings_list = []  # Clear memory
#             print(f"Checkpoint saved at batch {i // batch_size + 1}")

#     # Final save
#     np.save(os.path.join(SAVE_DIR, "final_embeddings.npy"), np.vstack(embeddings_list))
#     print("Final embeddings saved!")

#     return os.path.join(SAVE_DIR, "final_embeddings.npy")

In [ ]:
# full_texts = bert_df['cleaned_essays'].fillna("").tolist()
# embedding_file = bert_embeddings_colab(full_texts, batch_size=8)

In [ ]:
# print(f"Total rows in DataFrame: {len(bert_df)}")
# print(f"Total embeddings loaded: {len(final_embeddings)}")

In [ ]:
# !rm -rf "/content/drive/My Drive/bert_embeddings/*"

# # Re-run embeddings with fresh save
# embedding_file = bert_embeddings_colab(full_texts, batch_size=8)  # Regenerate all embeddings

In [ ]:
# 1. Find all embedding files
embedding_files = sorted(glob.glob(os.path.join(SAVE_DIR, "embeddings_batch_*.npy")))

# 2. Load and stack all saved embeddings
final_embeddings = np.vstack([np.load(f) for f in embedding_files])

# 3. Check the shape
print(f"Successfully loaded embeddings! Final shape: {final_embeddings.shape}")

# 4. Ensure shape matches DataFrame
if len(final_embeddings) > len(bert_df):
    final_embeddings = final_embeddings[:len(bert_df)]
elif len(final_embeddings) < len(bert_df):
    padding = np.zeros((len(bert_df) - len(final_embeddings), final_embeddings.shape[1]))
    final_embeddings = np.vstack([final_embeddings, padding])

# Convert NumPy array to list of lists
final_embeddings_list = final_embeddings.tolist()

# Merge into DataFrame
bert_df['bert_embeddings'] = final_embeddings_list

print(f"Embeddings successfully merged into DataFrame! Final shape: {bert_df.shape}")

Successfully loaded embeddings! Final shape: (58800, 384)
Embeddings successfully merged into DataFrame! Final shape: (58822, 2)


In [ ]:
bert_df.sample(5)

,cleaned_essays,bert_embeddings
33779,driven complex caring getting groove making pe...,"[-0.011404311284422874, -0.02866128645837307, ..."
18511,recently finished phd uc berkeley also went un...,"[-0.004183593671768904, -0.09250206500291824, ..."
33399,let grab drink message youre sf 3 sf_pisces wo...,"[-0.00020296289585530758, -0.03991928324103355..."
52583,originally cold far away place ive sf twelve y...,"[0.04098859429359436, -0.06496076285839081, 0...."
54637,part flamboyant extrovert part gentle nerd im ...,"[-0.013959165662527084, -0.04244036227464676, ..."


In [ ]:
df['bert_embeddings'] = bert_df['bert_embeddings']

In [ ]:
print(type(df['bert_embeddings'][0]))
print(df['bert_embeddings'][0][:5])

<class 'list'>
[-0.05193902179598808, -0.05350712314248085, 0.01932149939239025, -0.05699744448065758, 0.0025898332241922617]


In [ ]:
df.sample(10)

,age,status,sex,orientation,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,smokes,speaks,speaks_original,combined_essay_cols,bert_embeddings
33708,39,single,m,straight,unknown,socially,unknown,law school_graduated,asian,68.0,entertainment / media,"san francisco, california",unknown,likes pets,other,no,multilingual,"english, spanish, korean, japanese","i am discreetly optimistic, surprisingly sport...","[-0.045035649091005325, -0.020917754620313644,..."
34397,57,single,m,straight,unknown,socially,never,high school_graduated,mixed,71.0,executive / management,"oakland, california",unknown,has pets,other,no,monolingual,english,i love the northwest and often spend a lot of ...,"[-0.04473533108830452, -0.0551692470908165, 0...."
19246,29,single,m,straight,unknown,socially,unknown,unknown,asian,65.0,banking / financial / real estate,"hayward, california","no Kids, might want",unknown,other,no,monolingual,english,i can make you laugh! i can make you smile! i ...,"[0.049605581909418106, -0.091891810297966, 0.0..."
10658,24,single,f,straight,anything,socially,never,college/university_studying,white,66.0,science / tech / engineering,"burlingame, california","no Kids, might want",has pets,catholicism,no,multilingual,"english, spanish, italian",i'm really horrible at summarizing myself... i...,"[-0.04854567348957062, -0.0801982432603836, 0...."
38298,26,single,m,straight,anything,often,never,masters_graduated,asian,69.0,sales / marketing / biz dev,"san francisco, california","no kids, doesn't want",likes pets,atheism,occasionally,multilingual,"english, chinese, spanish",i am the psychic spy from china that the red h...,"[-0.002872900106012821, -0.049166470766067505,..."
36751,36,single,f,straight,unknown,socially,unknown,ph.d_studying,white,65.0,education / academia,"oakland, california",unknown,likes dogs,judaism,no,multilingual,"english, hebrew",i walk fast and talk fast like i'm still back ...,"[-0.0048864237032830715, -0.00146183290053159,..."
35149,30,single,f,straight,unknown,socially,unknown,masters_graduated,white,69.0,education / academia,"san francisco, california",unknown,unknown,other,no,multilingual,"english, spanish",i love to laugh and make people laugh. i want ...,"[0.04360361024737358, -0.07708050310611725, 0...."
31520,34,married,f,straight,unknown,socially,never,college/university_graduated,white,68.0,political / government,"emeryville, california",unknown,has pets,christianity,no,multilingual,"french, english",lest you take a look at my location and think ...,"[0.0191437266767025, -0.06858059018850327, 0.0..."
56593,20,single,m,straight,anything,socially,never,two-year college_studying,mixed,71.0,student,"vallejo, california",unknown,unknown,agnosticism,no,monolingual,english,i'll get into this later i am currently trying...,"[0.04689529165625572, -0.17005734145641327, 0...."
13630,27,single,f,straight,anything,socially,unknown,law school_studying,white,66.0,student,"san francisco, california",unknown,likes dogs,other,regularly,multilingual,"english, croatian, italian, other, serbian",there are two things that i want out of life: ...,"[-0.03750823065638542, -0.14278817176818848, -..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58822 entries, 0 to 58821
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  58822 non-null  int64  
 1   status               58822 non-null  object 
 2   sex                  58822 non-null  object 
 3   orientation          58822 non-null  object 
 4   diet                 58822 non-null  object 
 5   drinks               58822 non-null  object 
 6   drugs                58822 non-null  object 
 7   education            58822 non-null  object 
 8   ethnicity            58822 non-null  object 
 9   height               58822 non-null  float64
 10  job                  58822 non-null  object 
 11  location             58822 non-null  object 
 12  offspring            58822 non-null  object 
 13  pets                 58822 non-null  object 
 14  religion             58822 non-null  object 
 15  smokes               58822 non-null 

In [ ]:
df.to_csv('matchmaking_bert_embeddings_v1.csv', index=False)